In [ ]:
import string
import numpy as np
import os
import xml.etree.ElementTree as ET
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from transformers import DataCollatorWithPadding
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
import torch.optim as optim
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    text = root.find('TEXT').text.strip()
    tags = {tag.tag: tag.attrib['met'] for tag in root.find('TAGS')}
    return text, tags

xml_dir = 'part1'
data = []

for file_name in os.listdir(xml_dir):
    if file_name.endswith('.xml'):
        file_path = os.path.join(xml_dir, file_name)
        text, tags = parse_xml(file_path)
        filtered_tags = {key: tags[key] for key in ['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES']}
        filtered_tags['text'] = text
        data.append(filtered_tags)

df = pd.DataFrame(data)

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation.replace('-', '')))
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

df['clean_text'] = df['text'].apply(preprocess_text)

df['ABDOMINAL'] = df['ABDOMINAL'].apply(lambda x: 1 if x == 'met' else 0)
df['CREATININE'] = df['CREATININE'].apply(lambda x: 1 if x == 'met' else 0)
df['MAJOR-DIABETES'] = df['MAJOR-DIABETES'].apply(lambda x: 1 if x == 'met' else 0)

labels = df[['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES']].values
texts = df['clean_text'].tolist()

tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')

def tokenize_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

class MedicalDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.texts.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def train_and_evaluate(train_loader, val_loader, model, optimizer, scheduler, device, epochs=15, accumulation_steps=4):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()  # Move optimizer.zero_grad() outside the batch loop

        for step, batch in enumerate(train_loader):
            # Move the batch to the device
            labels = batch.pop('labels').to(device)
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss = loss / accumulation_steps

            loss.backward()

            if (step + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                scheduler.step()

        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_train_loss}")

    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            labels = batch.pop('labels').to(device)
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch, labels=labels)
            logits = outputs.logits

            predictions.append(logits.cpu().numpy())
            true_labels.append(labels.cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    pred_labels = (predictions > 0.5).astype(int)

    return classification_report(true_labels, pred_labels, target_names=['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES']), multilabel_confusion_matrix(true_labels, pred_labels)

model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=3)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
device = torch.device('cuda')

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

y = np.array([np.argmax(label) for label in labels])

fold_results = []

for fold, (train_val_index, test_index) in enumerate(kf.split(texts, y)):
    print(f"Fold {fold + 1}")

    train_val_texts = [texts[i] for i in train_val_index]
    test_texts = [texts[i] for i in test_index]
    train_val_labels = labels[train_val_index]
    test_labels = labels[test_index]

    train_texts, val_texts, train_labels, val_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.1111, random_state=42)  # 0.1111 * 90% = 10%

    train_encoded_texts = tokenize_texts(train_texts)
    val_encoded_texts = tokenize_texts(val_texts)
    test_encoded_texts = tokenize_texts(test_texts)

    train_dataset = MedicalDataset(train_encoded_texts, train_labels)
    val_dataset = MedicalDataset(val_encoded_texts, val_labels)
    test_dataset = MedicalDataset(test_encoded_texts, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=data_collator)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=data_collator)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=data_collator)

    total_steps = len(train_loader) * 15  # 15 epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=total_steps//10, num_training_steps=total_steps)

    fold_report, fold_confusion_matrices = train_and_evaluate(train_loader, val_loader, model, optimizer, scheduler, device)
    fold_results.append((fold_report, fold_confusion_matrices))

for i, (report, conf_matrices) in enumerate(fold_results):
    print(f"Results for Fold {i+1}:")
    print("Classification Report:")
    print(report)
    print("Confusion Matrices:")
    for j, label in enumerate(['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES']):
        print(f"Confusion Matrix for {label}:")
        print(conf_matrices[j])

model_save_path = 'path_to_save_final_model_after_cv'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = batch['labels'].cpu().numpy()
        predictions.append(logits)
        true_labels.append(label_ids)

predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_labels = (predictions > 0.5).astype(int)

conf_matrices = multilabel_confusion_matrix(true_labels, pred_labels)
print("Confusion Matrices:")
for i, label in enumerate(['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES']):
    print(f"Confusion Matrix for {label}:")
    print(conf_matrices[i])

class_report = classification_report(true_labels, pred_labels, target_names=['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES'])
print("Classification Report:")
print(class_report)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 10
Epoch 1, Loss: 0.005971172358840704
Epoch 2, Loss: 0.00618156767450273
Epoch 3, Loss: 0.005897634895518422
Epoch 4, Loss: 0.005803555327778061
Epoch 5, Loss: 0.006406250720222791
Epoch 6, Loss: 0.006001800609131654
Epoch 7, Loss: 0.006273890224595864
Epoch 8, Loss: 0.0067635606198261184
Epoch 9, Loss: 0.006133409605051081
Epoch 10, Loss: 0.00545814687696596
Epoch 11, Loss: 0.005751791410148144
Epoch 12, Loss: 0.005824444505075614
Epoch 13, Loss: 0.006568614936744173
Epoch 14, Loss: 0.005411131074652076
Epoch 15, Loss: 0.0058590020053088665


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Results for Fold 1:
Classification Report:
                precision    recall  f1-score   support

     ABDOMINAL       0.00      0.00      0.00         7
    CREATININE       0.00      0.00      0.00         8
MAJOR-DIABETES       0.00      0.00      0.00        11

     micro avg       0.00      0.00      0.00        26
     macro avg       0.00      0.00      0.00        26
  weighted avg       0.00      0.00      0.00        26
   samples avg       0.00      0.00      0.00        26

Confusion Matrices:
Confusion Matrix for ABDOMINAL:
[[14  0]
 [ 7  0]]
Confusion Matrix for CREATININE:
[[13  0]
 [ 8  0]]
Confusion Matrix for MAJOR-DIABETES:
[[10  0]
 [11  0]]
Results for Fold 2:
Classification Report:
                precision    recall  f1-score   support

     ABDOMINAL       0.50      0.11      0.18         9
    CREATININE       0.00      0.00      0.00        10
MAJOR-DIABETES       0.62      0.45      0.53        11

     micro avg       0.60      0.20      0.30        30
  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
ls

part1.zip  sample_data/


In [ ]:
!unzip part1.zip -d .

Archive:  part1.zip
   creating: ./part1/
  inflating: ./part1/100.xml         
  inflating: ./part1/101.xml         
  inflating: ./part1/102.xml         
  inflating: ./part1/103.xml         
  inflating: ./part1/104.xml         
  inflating: ./part1/105.xml         
  inflating: ./part1/106.xml         
  inflating: ./part1/107.xml         
  inflating: ./part1/109.xml         
  inflating: ./part1/110.xml         
  inflating: ./part1/111.xml         
  inflating: ./part1/112.xml         
  inflating: ./part1/113.xml         
  inflating: ./part1/114.xml         
  inflating: ./part1/116.xml         
  inflating: ./part1/117.xml         
  inflating: ./part1/121.xml         
  inflating: ./part1/122.xml         
  inflating: ./part1/123.xml         
  inflating: ./part1/124.xml         
  inflating: ./part1/125.xml         
  inflating: ./part1/126.xml         
  inflating: ./part1/127.xml         
  inflating: ./part1/128.xml         
  inflating: ./part1/129.xml         
  inflat

In [ ]:
ls

part1/  part1.zip  path_to_save_final_model_after_cv/  sample_data/


In [ ]:
zip path_to_save_final_model_after_cv/

SyntaxError: invalid syntax (<ipython-input-4-8e1b91c98971>, line 1)

In [ ]:
!zip out.zip path_to_save_final_model_after_cv/

  adding: path_to_save_final_model_after_cv/ (stored 0%)


In [ ]:
ls

out.zip  part1/  part1.zip  path_to_save_final_model_after_cv/  sample_data/


In [ ]:
cd path_to_save_final_model_after_cv/

/content/path_to_save_final_model_after_cv


In [ ]:
ls -l


total 423344
-rw-r--r-- 1 root root       884 Jun 26 20:43 config.json
-rw-r--r-- 1 root root 433273844 Jun 26 20:43 model.safetensors
-rw-r--r-- 1 root root       125 Jun 26 20:43 special_tokens_map.json
-rw-r--r-- 1 root root      1243 Jun 26 20:43 tokenizer_config.json
-rw-r--r-- 1 root root    213450 Jun 26 20:43 vocab.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

config.json  model.safetensors  special_tokens_map.json  tokenizer_config.json  vocab.txt


In [ ]:
!cp model.safetensors ../drive/MyDrive/